## Asynchronous Flow

Synchronous from outside, Jina runs asynchronously underneath: it manages the eventloop(s) for scheduling the jobs. In some scenario, user wants more control over the eventloop, then `AsyncFlow` comes to use. In the example below, Jina is part of the integration where another heavy-lifting job is running concurrently:

In [ ]:
#Run this command to install Jina in this notebook 
!pip install jina

In [ ]:
import asyncio
from jina import AsyncFlow

async def run_async_flow_5s():  # WaitDriver pause 5s makes total roundtrip ~5s
    with AsyncFlow().add(uses='- !WaitDriver {}') as f:
        await f.index_ndarray(numpy.random.random([5, 4]), on_done=print)

async def heavylifting():  # total roundtrip takes ~5s
    print('heavylifting other io-bound jobs, e.g. download, upload, file io')
    await asyncio.sleep(5)
    print('heavylifting done after 5s')

async def concurrent_main():  # about 5s; but some dispatch cost, can't be just 5s, usually at <7s
    await asyncio.gather(run_async_flow_5s(), heavylifting())

`AsyncFlow` is very useful when using Jina inside Jupyter Notebook. As Jupyter/ipython already manages an eventloop and thanks to [`autoawait`](https://ipython.readthedocs.io/en/stable/interactive/autoawait.html), the following code can run out-of-the-box in Jupyter:

In [ ]:
from jina import AsyncFlow
import numpy

with AsyncFlow().add() as f:
    await f.index_ndarray(numpy.random.random([5, 4]), on_done=print)